# Installs

In [ ]:
!nvidia-smi

Fri Nov 18 23:31:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb -q
import wandb

     |████████████████████████████████| 1.9 MB 34.0 MB/s 
     |████████████████████████████████| 182 kB 86.5 MB/s 
     |████████████████████████████████| 168 kB 80.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 96.5 MB/s 
     |████████████████████████████████| 166 kB 95.1 MB/s 
     |████████████████████████████████| 162 kB 101.2 MB/s 
     |████████████████████████████████| 162 kB 92.8 MB/s 
     |████████████████████████████████| 158 kB 98.7 MB/s 
     |████████████████████████████████| 157 kB 99.9 MB/s 
     |████████████████████████████████| 157 kB 99.6 MB/s 
     |████████████████████████████████| 157 kB 102.2 MB/s 
     |████████████████████████████████| 157 kB 97.2 MB/s 
     |████████████████████████████████| 157 kB 100.5 MB/s 
     |████████████████████████████████| 157 kB 100.6 MB/s 
     |████████████████████████████████| 157 kB 99.4 MB/s 
     |████████████████████████████████| 156 kB 97.7 MB/s 


In [ ]:
wandb.login(key="6b14f00e0515c5740d7e07042926608efdc6e526")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = train_ ### Wandb Config for your run
)

NameError: ignored

## Levenshtein

This may take a while

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 43.8 MB/s 
     |████████████████████████████████| 2.2 MB 100.1 MB/s 
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 22.35 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/

## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import csv
import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Dataset and Dataloader

In [ ]:
CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:41]
mapping = CMUdict_ARPAbet
LABELS = ARPAbet[:41]
print(len(LABELS))


41


### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    #TODO
    def __init__(self, partition= "train_tiny"): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''


        if partition=="train_tiny":
          data_path='/content/hw3p2/train-clean-100'
        elif partition=="train_small":
          data_path='/content/hw3p2/train-clean-360'
        else:
          data_path='/content/hw3p2/dev-clean'
        self.mfcc_dir = data_path+'/mfcc'
        self.transcript_dir = data_path+'/transcript/raw'


        mfcc_names = (os.listdir(self.mfcc_dir))
        transcript_names =(os.listdir(self.transcript_dir))
        mfcc_names.sort()
        transcript_names.sort()


        # Creating Array

        self.mfccs, self.transcripts = [], []
        self.PHONEMES = PHONEMES
        def helper(x):
          return self.PHONEMES.index(x)

       
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
       
            mfcc =np.load(self.mfcc_dir+"/"+mfcc_names[i],allow_pickle=True)
            
            transcript = np.load(self.transcript_dir+"/"+transcript_names[i],allow_pickle=True)
            transcript.reshape(1,transcript.shape[0])
            transcript=transcript[(transcript != "[SOS]") & (transcript != "[EOS]")]
            transcript=np.vectorize(helper)(transcript)
           

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
        self.length = len(mfcc_names)

    def __len__(self):
        
        return self.length
       

    def __getitem__(self, ind):
      
        mfcc =  self.mfccs[ind]
        transcript =self.transcripts[ind]
        return mfcc, transcript


    def collate_fn(self,batch):
        batch_mfcc =[torch.from_numpy(x[0]) for x in batch]
        batch_transcript = [torch.from_numpy(x[1]) for x in batch]
        batch_mfcc_pad = pad_sequence(batch_mfcc,batch_first=True)
        batch_transcript_pad = pad_sequence(batch_transcript,batch_first=True)
        lengths_mfcc =[ len(x) for x in batch_mfcc]
        lengths_transcript =[len(x) for x in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self,data_path): 
        self.data_path =data_path
        mfcc_names = (os.listdir(self.data_path))
        mfcc_names.sort() 
        self.mfccs=[]


        for i in range(0, len(mfcc_names)):
            mfcc =np.load(self.data_path+"/"+mfcc_names[i],allow_pickle=True)

            self.mfccs.append(mfcc)

        self.length = len(self.mfccs)
    def __len__(self):
      return self.length

    def __getitem__(self, ind):
      return self.mfccs[ind]


    def collate_fn(self,batch):
        batch_mfcc =[torch.from_numpy(x) for x in batch]
        batch_mfcc_pad = pad_sequence(batch_mfcc,batch_first=True)
      
        lengths_mfcc =[len(x) for x in batch_mfcc]
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)


### Data - Hyperparameters

In [ ]:
config = {
    "beam_width" : 30,
    "lr" : 2e-3,
    "epochs" : 70,
    "batch_size":48
    } # Feel free to add more items here

In [ ]:
 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()


88

In [ ]:

# Do NOT forget to pass in the collate function as parameter while creating the dataloader



train_data = AudioDataset("train_tiny") 
val_data =  AudioDataset("validation")
test_data = AudioTestDataset("/content/hw3p2/test-clean/mfcc")



In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, num_workers= 8, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False,collate_fn=test_data.collate_fn)


train_loader = torch.utils.data.DataLoader(train_data, num_workers= 8,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True,collate_fn=train_data.collate_fn)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 4,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False,collate_fn=val_data.collate_fn)



print("Batch size: ", config["batch_size"])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 



torch.Size([64, 1657, 15]) torch.Size([64, 223]) torch.Size([64]) torch.Size([64])


# Model Config

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

41

## Basic

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.embed_size=15
        self.hidden_size=680
        self.embedding = torch.nn.Sequential(
            nn.Conv1d(self.embed_size, self.hidden_size, 3, padding=1, bias=False),
            nn.BatchNorm1d(self.hidden_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(self.hidden_size, self.hidden_size, 3, padding=1, bias=False),
            nn.BatchNorm1d(self.hidden_size),
            nn.ReLU(inplace=True))
        self.lstm = nn.LSTM(input_size=self.hidden_size,
                            hidden_size=self.hidden_size,
                            num_layers=3,
                            bias=True,
                            dropout=0.12,
                            bidirectional=True) 
       
        self.classification = nn.Sequential( torch.nn.Sequential(
            nn.Linear(self.hidden_size*2, self.hidden_size*2),
            nn.Linear(self.hidden_size*2,OUT_SIZE))
        )
        self.logSoftmax =nn.LogSoftmax(dim=2)

    def forward(self, x, lx):
        x_cnn_input = x.permute(0, 2, 1) 
        x_post_cnn = self.embedding(x_cnn_input)
       
        x_rnn_input = x_post_cnn.permute(2, 0, 1)


        packed = pack_padded_sequence(x_rnn_input, lx,enforce_sorted=False)
        output,hidden=self.lstm(packed)
        out, out_lens = pad_packed_sequence(output, batch_first=True)
        prob=self.classification(out)
        prob=self.logSoftmax(prob)
        prob = prob.permute(1, 0, 2)
        return prob, out_lens



        

## INIT

# Training Config

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
checkpoint =torch.load('/content/drive/MyDrive/11-785-s22/hw/lstm_360_512_4.pth')

model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [ ]:
summary(model,x.to(device),lx)

NameError: ignored

In [ ]:



criterion = torch.nn.CTCLoss(blank=0)
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html


optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'],weight_decay=5e-5)

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
decoder = CTCBeamDecoder(
    LABELS,
    beam_width=config["beam_width"],
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)


scheduler =torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)


scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    # TODO: look at docs for CTC.decoder and find out what is returned here
    h=torch.transpose(h, 0, 1)
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(h, seq_lens = lh)

    

    batch_size=len(h)
    distance = 0 # Initialize the distance to be 0 initially

  

  

    for i in range(batch_size): 
      

      sliced=beam_results[i, 0, : out_lens[i][0]]
      cur_string="".join([labels[x] for x in sliced])
     
      cur_y="".join([labels[n] for n in y[i]])
      
      distance+=Levenshtein.distance(cur_string, cur_y[:ly[i]])
    
    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    
    return distance

In [ ]:

with torch.no_grad():
  for i, data in enumerate(train_loader):
      
     
      x,y,lx,ly=data
      x,y,lx,ly=x.to(device),y.to(device),lx,ly
     
      out,predict_length=model(x,lx)
      
 
      loss = criterion(out,y,predict_length, ly)
      print(f"loss: {loss}")

      distance = calculate_levenshtein(out, y,predict_length, ly, decoder, LABELS, debug = False)
      print(f"lev-distance: {distance}")

      break

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
torch.cuda.empty_cache()
def evaluate(data_loader, model):
    
    dist = 0
    train_loss = 0

    decoder= CTCBeamDecoder(LABELS, beam_width=config["beam_width"],num_processes=4,blank_id=0,log_probs_input=True)
    model.eval()
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    for i, (data) in enumerate(data_loader):
       
        x,y,lx,ly=data
        x,y,lx,ly=x.to(device),y.to(device),lx,ly
     
        out,predict_length=model(x,lx)
        
        # Get model outputs
        with torch.inference_mode():
          outputs,predict_length=model(x,lx)
          loss = criterion(outputs,y,predict_length, ly)

        train_loss += float(loss.item())
        dist+= calculate_levenshtein(outputs, y,predict_length, ly, decoder, LABELS, debug = False)

       

        batch_bar.set_postfix(
            acc="{:.04f}".format(float(dist/ (i + 1))),
            loss="{:.04f}".format(float(train_loss / (i + 1))))
        

        batch_bar.update()
        del x 
        del lx
        del y
        del ly 
        gc.collect()
        torch.cuda.empty_cache()
        
    batch_bar.close()
    dist = float(dist / len(data_loader))
    total_loss = float(train_loss / len(data_loader))
    
    
    return total_loss, dist

### Training Setup

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:

## we might need to place scheduler somewhere in the middle

def train_step(train_loader, model, optimizer, criterion,scaler):
   
   
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        x,y,lx,ly=data
        x,y,lx,ly=x.to(device),y.to(device),lx,ly
        out,predict_length=model(x,lx)
     
        with torch.cuda.amp.autocast(): 
          
          outputs,predict_length=model(x,lx)
          loss = criterion(outputs,y,predict_length, ly)

        train_loss += float(loss.item())

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(train_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() 
        batch_bar.update()
        del x
        del y
        del lx 
        del ly
        gc.collect()
        torch.cuda.empty_cache()
    
    batch_bar.close()
    train_loss /= float(train_loss/ len(train_loader))
    

    return train_loss

### Train Loop

In [ ]:
torch.cuda.empty_cache()
gc.collect()

best_val_dist = checkpoint['val_dist']
print(best_val_dist)
for epoch in range(config["epochs"]):

    # one training step
    
    train_loss=train_step(train_loader,model, optimizer, criterion,scaler)
    if True:
      val_loss, val_dist=evaluate(val_loader,model)
      if True:
        scheduler.step(val_dist)
      print("Val Dist {:.04f}%\t Val Loss {:.04f}".format(val_dist, val_loss))
      if val_dist < best_val_dist:
        print("Saving model")
        torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'val_dist': val_dist, 
                  'epoch': epoch}, '/content/drive/MyDrive/11-785-s22/hw/lstm_360_512_4_fine_tune.pth')
        best_val_dist = val_dist

    
    
     
    

    # You may want to log some hyperparameters and results on wandb
    

4.795542635658915


Train:   0%|          | 0/446 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
torch.cuda.empty_cache()
gc.collect()

101

True

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder


def make_output(h, lh):
    

    decoder= CTCBeamDecoder(LABELS, beam_width=config["beam_width"]+20,num_processes=4,blank_id=0,log_probs_input=True)
    h=torch.transpose(h, 0, 1)
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(h, seq_lens=lh) 
    batch_size = len(h)

    preds = []
    for i in range(batch_size): # Loop through each element in the batch
        h_sliced =beam_results[i, 0, : out_seq_len[i][0]]
        h_string ="".join([LABELS[x] for x in h_sliced])
        preds.append(h_string)
    return preds

In [ ]:
def predict(model,data_loader):
  model.eval()
  out=[]
  for i,data in enumerate(data_loader):
    x,lx=data
    x,lx=x.to(device),lx
    x,lx=model(x,lx)
    preds=make_output(x, lx)
    out+=preds
  return out
    
    
                                

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle

torch.cuda.empty_cache()
predictions = predict( model,test_loader)



In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/hw3p2/test-clean/transcript/random_submission.csv')
df.label = predictions

df.to_csv('submission.csv', index = False)
!kaggle competitions submit -c 11-785-f22-hw3p2 -f submission.csv -m "Message"

100% 209k/209k [00:03<00:00, 60.3kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)